## 12.1 

In [ ]:
import threading
import time
def worker(msg):
    print("{} is start : {}".format(threading.current_thread().getName(), msg))
    time.sleep(2)
    print("{} is end".format(threading.current_thread().getName()))

for i in range(10):
    msg = "hello {}".format(i)
    th = threading.Thread(target=worker, name="[{}]".format(i), args=(msg,))
    th.start()

In [ ]:
class Worker(threading.Thread):
    def __init__(self, args, name=""):
        threading.Thread.__init__(self)
        self.name = name
        self.args = args

    def run(self):
        print("{} is start : {}".format(threading.currentThread().getName(), self.args[0]))
        time.sleep(5)
        print("{} is end".format(threading.currentThread().getName()))

for i in range(10):
    msg = "hello"
    th = Worker(name="[th cls {}]".format(i), args=(msg,))
    th.start()  # run()에 구현한 부분이 실행된다
    th.join()

In [ ]:
if th.is_alive():
    print('Still running')
else:
    print('Completed')

- daemon

In [ ]:
def worker(msg):
    print("{} is start : {}.".format(threading.currentThread().getName(), msg))
    time.sleep(3)
    print("{} is end".format(threading.currentThread().getName()))

def main():
    msg = 'hello'
    th = threading.Thread(target=worker, daemon=True, args=(msg,))
    th.start()
    print("Main thread End")
    

if __name__ == "__main__":
    main()

In [ ]:
def main():
    msg = 'hello'
    th = threading.Thread(target=worker, daemon=True, args=(msg,))
    th.start()
    th.join()
    print("Main thread End")
                                                                        
if __name__ == "__main__":
    main()

- 종료 지시

In [ ]:
from threading import Thread
class CountdownTask:
    def __init__(self):
        self._running = True
        
    def terminate(self):
        self._running = False
        
    
    def run(self, n):
        while self._running and n>0 :
            print('T-minus', n)
            n -= 1
            time.sleep(1)
c = CountdownTask()
t = Thread(target=c.run, args=(5,))
t.start()

In [ ]:
c.terminate()

## 12.5

In [4]:
## main thread
counter_lock = threading.Lock()
counter = 0
...

## worker thread A
counter_lock.acquire()
counter += 1
counter_lock.release()

## worker thread B
counter_lock.acquire()
print(counter)
counter_lock.release()

1


In [1]:
import threading
from contextlib import contextmanager

_local = threading.local() #스레드가 다른 스레드와 공유하지 않고 자신의 스코프 내에서만 사용할 변수를 여기에 정의할 수 있음

@contextmanager
def acquire(*locks):
    # 객체 식별자로 락 정렬 ***핵심***
    locks = sorted(locks, key=lambda x:id(x))
    
    #이미 취득한 락의 순서가 깨지지 않도록 주의.
    acquired=getattr(_local, 'acquired', [])
    if acquired and max(id(lock) for lock in acquired) >= id(locks[0]):
        raise RuntimeError('Lock order violation')
        
    acquired.extend(locks)
    _local.acquired = acquired
    
    try:
        for lock in locks:
            lock.acquire() #미리 취득한 락 확인하고 기존 락이 새롭게 취득하려는 객체 id보다 작은 값을 갖도록 강제.
        yield
    
    #취득한 반대 순서로 락 해제
    finally:
        for lock in reversed(locks):
            lock.release()
        del acquired[-len(locks):]

In [2]:
x_lock = threading.Lock() # 락 객체를 얻음
y_lock = threading.Lock()

def thread_1():
    n = 100
    while n > 0:
        with acquire(x_lock, y_lock):
            print('Thread-1')
            n -= 1
            
def thread_2():
    m = 100
    while m > 0:
        with acquire(y_lock ,x_lock):
            print('Thread-2')
            m -= 1
            
t1 = threading.Thread(target=thread_1, daemon=True)
t1.start()

t2 = threading.Thread(target=thread_2, daemon=True)
t2.start()

Thread-1
Thread-1
Thread-1
Thread-1
Thread-1
Thread-1
Thread-1
Thread-1
Thread-1
Thread-1
Thread-1
Thread-1
Thread-1
Thread-1
Thread-1
Thread-1
Thread-1
Thread-1
Thread-1
Thread-1
Thread-1
Thread-1
Thread-2
Thread-1
Thread-2
Thread-2
Thread-1
Thread-2
Thread-1
Thread-2
Thread-1
Thread-2
Thread-1
Thread-2
Thread-1
Thread-2
Thread-1
Thread-2
Thread-1
Thread-2
Thread-1
Thread-2
Thread-1
Thread-2
Thread-1
Thread-2
Thread-1
Thread-2
Thread-1
Thread-2
Thread-1
Thread-2
Thread-1
Thread-2
Thread-1
Thread-2
Thread-1
Thread-2
Thread-1
Thread-2
Thread-1
Thread-2
Thread-1
Thread-2
Thread-1
Thread-2
Thread-1
Thread-2
Thread-1
Thread-2
Thread-1
Thread-2
Thread-1
Thread-2
Thread-1
Thread-2
Thread-1
Thread-2
Thread-1
Thread-2
Thread-1
Thread-2
Thread-1
Thread-2
Thread-1
Thread-2
Thread-1
Thread-2
Thread-1
Thread-2
Thread-1
Thread-2
Thread-1
Thread-2
Thread-1
Thread-2
Thread-1
Thread-2
Thread-1
Thread-2
Thread-1
Thread-1
Thread-1
Thread-1
Thread-1
Thread-1
Thread-1
Thread-1
Thread-2
Thread-1
Thread-2
T

In [ ]:
class acquire(object):
    def __init__(self,*locks):
        self.locks = sorted(locks, key=lambda x: id(x))
    def __enter__(self):
        for lock in self.locks:
            lock.acquire()
    def __exit__(self,ty,val,tb):
        for lock in reversed(self.locks):
            lock.release()
        return False

## 12.9 gil 다루기

## 12.13 다중 스레드 큐 폴링

In [6]:
import queue
import socket
import os
class PollableQueue(queue.Queue):
    def __init__(self):
        super().__init__()
        if os.name == 'posix':
            self._putsocket, self._getsocket = socket.socketpair()
        else:
            server = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            server.bind(('127.0.0.1', 0))
            server.listen(1)
            self._putsocket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            self._putsocket.connect(server.getsockname())
            self._getsocket, _ = server.accept()
            server.close()
    def fileno(self):
        return self._getsocket.fileno()
    
    def put(self, item):
        super().put(item)
        self._putsocket.send(b'x')

    def get(self):
        self._getsocket.recv(1)
        return super().get()

In [7]:
import select
import threading
def consumer(queues):
    while True:
        can_read, _, _ = select.select(queues, [], [])
        for r in can_read:
            item = r.get()
            print('Got:', item)

q1 = PollableQueue()
q2 = PollableQueue()
q3 = PollableQueue()
t = threading.Thread(target=consumer, args=([q1, q2, q3],), daemon=True)
t.start()

q1.put(1)
q2.put(10)
q3.put('hello')
q2.put('anything')


Got: 1
Got: 10
Got: hello
Got: anything
